In [ ]:
import os

import pandas as pd
import torch
import torchvision.io as io
import torchvision.transforms as transforms
from googletrans import Translator
from tqdm.auto import tqdm

%load_ext autoreload
%autoreload 2

In [12]:
data_path = "/content/drive/MyDrive/UCU/CV/HW5_Project/data/"
data_path = "data"
ann_path = os.path.join(data_path, "validation_annotation.csv")
annotation_df = pd.read_csv(ann_path)

In [30]:
any([v is None for v in annotation_df.basestyle.values])

False

In [31]:
class HairStyleDataset(torch.utils.data.Dataset):
    def __init__(self, annotation_df, transform=None):
        # translator = Translator(service_urls=["translate.googleapis.com"])
        # self.img_labels = [
        #     translator.translate(label).text for label in tqdm(annotation_df.basestyle.values)
        # ]
        self.img_labels = annotation_df.basestyle.values
        self.img_labels, self.label_encoding = pd.factorize(self.img_labels)
        self.img_dir_path = annotation_df.img_dir_path
        self.filename = annotation_df.filename
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        image_path = os.path.join(self.img_dir_path[idx], self.filename[idx])
        image = io.imread(image_path)

        if self.transform:
            image = self.transform(image)

        label = self.img_labels[idx]

        return image, label

In [32]:
def decode_label(label, label_encoding):
    return label_encoding[label]

In [33]:
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

dataset = HairStyleDataset(annotation_df, transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18

# Load the ResNet18 model and replace its classifier head
model = resnet18(pretrained=True)
model.fc = nn.Linear(512, num_classes)

# Set the model to train mode
model.train()

# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# Train the model for a number of epochs
for epoch in range(num_epochs):
    for i, data in enumerate(tqdm(dataloader)):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

In [ ]:
import netron
from ultralytics import YOLO

In [ ]:
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6.23M/6.23M [00:04<00:00, 1.41MB/s]



[Adding "Freeze layers" for transfer learning](https://github.com/ultralytics/ultralytics/issues/562)

In [ ]:
model.info(verbose=True)

YOLOv8n summary: 225 layers, 3157200 parameters, 0 gradients, 8.9 GFLOPs


layer                                     name  gradient   parameters                shape         mu      sigma
    0                      model.0.conv.weight     False          432        [16, 3, 3, 3]   -0.00279      0.152
    1                        model.0.bn.weight     False           16                 [16]       2.97       1.86
    2                          model.0.bn.bias     False           16                 [16]      0.249       4.17
    3                      model.1.conv.weight     False         4608       [32, 16, 3, 3]   -0.00012      0.063
    4                        model.1.bn.weight     False           32                 [32]       5.02       1.12
    5                          model.1.bn.bias     False           32                 [32]      0.942        1.5
    6                  model.2.cv1.conv.weight     False         1024       [32, 32, 1, 1]     -0.011     0.0906
    7                    model.2.cv1.bn.weight     False           32                 [32]      

In [ ]:
model.export(format="onnx")

Ultralytics YOLOv8.0.17 🚀 Python-3.10.9 torch-2.0.0.dev20221228 CPU
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from yolov8n.pt with output shape (1, 84, 8400) (6.2 MB)

ONNX: starting export with onnx 1.13.0...
ONNX: export success ✅ 0.3s, saved as yolov8n.onnx (12.2 MB)

Export complete (0.5s)
Results saved to /Users/astadnik/edu/masters/sem_3/CV/5/hairstyle_detector
Predict:         yolo task=detect mode=predict model=yolov8n.onnx -WARNING ⚠️ not yet supported for YOLOv8 exported models
Validate:        yolo task=detect mode=val model=yolov8n.onnx -WARNING ⚠️ not yet supported for YOLOv8 exported models
Visualize:       https://netron.app


========== Diagnostic Run torch.onnx.export version 2.0.0.dev20221228 ==========
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
netron.start("yolov8n.onnx")

Serving 'yolov8n.onnx' at http://localhost:8080


('localhost', 8080)

[FINETUNING TORCHVISION MODELS](https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html)